# **TP3 Face détection**

In [2]:
import os
import cv2
from skimage import feature 
from sklearn.svm import SVC
import joblib
import numpy as np
import mtcnn 

c:\Users\ousse\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


In [3]:
# Face detection function
def detect_faces(image):
    detector = mtcnn.MTCNN()
    faces = detector.detect_faces(image)
    return faces

In [4]:
# Function to extract LBP features
def extract_lbp(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    lbp = feature.local_binary_pattern(gray, P=8, R=1, method="uniform")
    (hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, 11), range=(0, 10))
    hist = hist.astype("float")
    hist /= hist.sum()
    return hist

# Function to extract HOG features
def extract_hog(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    resized = cv2.resize(gray, (64, 64))  # Standardize size
    
    hog_features = feature.hog(resized, 
                      orientations=9,
                      pixels_per_cell=(8, 8),
                      cells_per_block=(2, 2),
                      block_norm='L2-Hys',
                      feature_vector=True)
    
    return hog_features

# Function to extract pooled SIFT features
def extract_sift_pooled(image, pooling='mean'):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(gray, None)
    
    if descriptors is None:
        return None  # Return None if no descriptors found
    
    # Pool descriptors into a fixed-size vector
    if pooling == 'mean':
        pooled_feature = np.mean(descriptors, axis=0)
    elif pooling == 'max':
        pooled_feature = np.max(descriptors, axis=0)
    else:
        raise ValueError("Pooling method not supported. Use 'mean' or 'max'.")
    
    return pooled_feature

In [5]:
# Function to extract features depending on the method
def extract_features(image, method):
    faces = detect_faces(image)
    if faces:
        # Assume the first detected face is the target
        x, y, width, height = faces[0]['box']
        face_region = image[y:y+height, x:x+width]
        
        if method == "lbp":
            feature = extract_lbp(face_region)
        elif method == "hog":
            feature = extract_hog(face_region)
        elif method == "sift":
            feature = extract_sift_pooled(face_region)  # Use pooled SIFT extraction
    else:
        # If no face detected, return None
        feature = None
    return feature

In [11]:
# Function to train the SVM model
def train_svm(features, labels):
    model = SVC(kernel='linear', probability=True)
    model.fit(features, labels)
    return model

In [12]:
def DataLoader_FeatuerExtract(path, method):
    profiles = os.listdir(path=path)
    features = []
    labels = []
    i = 0
    profiles_names = []

    for profi in profiles:
        profiles_names.append(profi)
        prof_path = os.path.join(path, profi) + "/"
        for img in os.listdir(prof_path):
            image_path = os.path.join(prof_path, img)
            image = cv2.imread(image_path)
            image_feature = extract_features(image, method)  
            
            if image_feature is not None: 
                features.append(image_feature)
                labels.append(i)
        i += 1
    
    return np.array(features), np.array(labels)

In [13]:
path = "./image_faciale_IM/"
Method = ["lbp", "sift", "hog"]
for method in Method:
    features, labels = DataLoader_FeatuerExtract(path, method)
    if len(np.unique(labels)) > 1:  # Vérifiez qu'il y a plus d'une classe
        model = train_svm(features, labels)
        joblib.dump(model, f"./Modele_tp3/face_recognition_model_{method}.pkl")
        print(f"Les poids du SVM basé sur {method} sont bien sauvegardés .... OK!")
    else:
        print(f"Attention : Le modèle {method} n'a qu'une seule classe. Vérifiez vos données.")

Les poids du SVM basé sur lbp sont bien sauvegardés .... OK!
Les poids du SVM basé sur sift sont bien sauvegardés .... OK!
Les poids du SVM basé sur hog sont bien sauvegardés .... OK!


In [7]:
import cv2
import joblib
path = "./image_faciale_IM/"
Method = ["lbp", "sift", "hog"]
cap = cv2.VideoCapture(0)

# Liste pour stocker les caractéristiques et les étiquettes
features = []
labels = []
name = os.listdir(path)
method = Method[2]  
# Charger le modèle sauvegardé correspondant à la méthode choisie
model_path = f"./Modele_tp3/face_recognition_model_{method}.pkl"
model = joblib.load(model_path)
# Boucle de traitement de la vidéo
while True:
    ret, frame = cap.read()
    if not ret:
        break
    # Détecter les visages
    faces = detect_faces(frame)
    for face in faces:
        x, y, width, height = face['box']
        detected_face = frame[y:y+height, x:x+width]

        # Extraire les caractéristiques du visage détecté
        if method == 'lbp':
            face_features = extract_lbp(detected_face)
        elif method == 'hog':
            face_features = extract_hog(detected_face)
        else:
            face_features = extract_sift_pooled(detected_face)
        # Vérifier si les caractéristiques ont été extraites avec succès
        if face_features is not None:
            # Prédiction avec le modèle SVM
            prediction = model.predict([face_features])
            
            cv2.rectangle(frame, (x, y), (x + width, y + height), (0, 255, 0), 2)
            cv2.putText(frame, name[prediction[0]], (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    cv2.imshow("Détection et Reconnaissance de Visage", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()